In [ ]:
%%writefile multisig.sl
contract multisig {
    address private signer0 = 0xDEADBEEF0;
    address private signer1 = 0xDEADBEEF1;
    address private signer2 = 0xDEADBEEF2;
    
    uint public counter = 0;
    address public lastSigner = 0;
    address public closingLastSinger = 0;
    address public pendingClosingDestination = 0;
    address public pendingTransactionDestination = 0;
    uint public pendingTransactionAmount = 0;

    function sendAmount(uint newCounterValue, 
                        address destination,
                        uint amount) 
    public returns (bool) {
        if ((msg.sender == signer0 || 
             msg.sender == signer1 ||
             msg.sender == signer2) && 
            address(this).balance >= amount) {
            // If nothing pending, 
            // make a new pending transaction
            if (counter + 1 == newCounterValue &&
                lastSigner == 0 &&
                pendingTransactionDestination == 0 &&
                pendingTransactionAmount == 0) {
                counter = newCounterValue;
                pendingTransactionDestination = destination;
                pendingTransactionAmount = amount;
                lastSigner = msg.sender;
                return true;
            }
            // If pending transaction, send and clear
            if (counter == newCounterValue &&
                lastSigner != msg.sender &&
                pendingTransactionDestination == destination &&
                pendingTransactionAmount == amount) {
                pendingTransactionDestination.send(pendingTransactionAmount);
                pendingTransactionDestination = 0;
                pendingTransactionAmount = 0;
                lastSigner = 0;
                return true;
            }    
            return false;
        }
        return false;
    }
  // TODO: Cancel Send Amount
    
  function closeWallet(uint newCounterValue, 
                       address destination)
    public returns (bool) {
        if (msg.sender == signer0 || 
            msg.sender == signer1 ||
            msg.sender == signer2) {
            if (counter + 1 == newCounterValue &&
                closingLastSinger == 0 &&
                pendingClosingDestination == 0) {
                counter = newCounterValue;
                pendingClosingDestination = destination;
                closingLastSinger = msg.sender;
                return true;
            }
            if (counter == newCounterValue &&
                closingLastSinger != msg.sender &&
                pendingClosingDestination == destination) {
                suicide(pendingClosingDestination);
                return true;
            }
        }
        return false;
    }
  // TODO: Cancel closeWallet
}

In [16]:
%%bash
solc --optimize=1 --binary stdout multisig.sl


======= multisig =======
Binary: 
640deadbeef06000819055640deadbeef16001819055640deadbeef26002819055600060038190556000600481905560006005819055600060068190556000600781905560006008819055610411806100596000396000f30060003560e060020a900480630ae8e1c214610063578063547d35d81461007e57806356f25b5e1461009057806361bc221a146100ab578063787c170d146100bd578063bed85a73146100d8578063bf173fe8146100f3578063cbde0bbd1461010e57005b61006b610135565b80600160a060020a031660005260206000f35b61008661014a565b8060005260206000f35b6100a1600435602435604435610150565b8060005260206000f35b6100b3610127565b8060005260206000f35b6100c561013c565b80600160a060020a031660005260206000f35b6100e061012e565b80600160a060020a031660005260206000f35b6100fb610143565b80600160a060020a031660005260206000f35b61011c6004356024356102e7565b8060005260206000f35b5b60035481565b5b60045481565b5b60055481565b5b60065481565b5b60075481565b5b60085481565b6000600054600160a060020a031633600160a060020a031614806101875750600154600160a060020a031633600160a060020a0316145b806